In [1]:
import numpy as np
import pandas as pd
import math
import sys

In [2]:
def extract_from_jscookies(cookies):
    set_cookies = cookies[["context_id", "host", "name", "value", "time_stamp", "record_type"]]
    
    # set cookies
    set_cookies.rename(columns={"record_type": "type"},inplace=True)
    set_cookies.fillna({'type':"set"}, inplace=True)
    return set_cookies

In [3]:
def sep_cookies(ls):
    res = []
    for l in ls:
        idx = l.find('=')
        if idx > 0:
            res.append((l[:idx], l[idx+1:]))
    return res


In [4]:
def extract_from_js(js):
    js_cookies = js[js.symbol == 'window.document.cookie']
    js_cookies = js_cookies[js_cookies.value.notnull()]
    js_cookies_list = list(map(lambda x:x.replace(" ","").split(';'), js_cookies.value.tolist()))
    js_cookies_ls_dict = list(map(lambda x:sep_cookies(x), js_cookies_list))

    js_cookies['cookies_dict'] = js_cookies_ls_dict

    js_cookies_temp = js_cookies[['context_id','script_url', 'cookies_dict', "time_stamp", "operation"]]

    ids = []
    hosts = []
    c_names = []
    c_values = []
    t_stamps = []
    types = []
    for _, row in js_cookies_temp.iterrows():
        Id = row.context_id
        host = row.script_url
        for name, value in row.cookies_dict:
            ids.append(Id)
            hosts.append(host)
            c_names.append(name)
            c_values.append(value)
            t_stamps.append(row.time_stamp)
            types.append(row.operation)

    js_cookies_new = pd.DataFrame({"context_id":ids, "host": hosts, "name":c_names, "value":c_values, "time_stamp":t_stamps, "type":types})

    # cookie use tables
    js_cookies_new.fillna({'type':"use"}, inplace=True)
    return js_cookies_new

In [5]:
def extract_http(httprequests):
    httprequests = httprequests[httprequests.cookies.notnull()]

    ids = []
    hosts = []
    c_names = []
    c_values = []
    t_stamps = []
    for i, row in httprequests.iterrows():
        Id = row.context_id
        cookies_ls = row.cookies.replace(' ','').split(';')
        for l in cookies_ls:
            idx = l.find('=')
            if idx > 0:
                ids.append(row.context_id)
                hosts.append(row.host)
                c_names.append(l[:idx])
                c_values.append(l[idx+1:])
                t_stamps.append(row.time_stamp)

    httpcookies = pd.DataFrame({"context_id":ids, "host": hosts, "name":c_names, "value":c_values, "time_stamp":t_stamps})
    httpcookies["type"] = ["sent"] * httpcookies.shape[0]
    #httpcookies.fillna({'type':"send"}, inplace=True)
    return httpcookies

In [6]:
def merge_cookies(jsc_cookies, js_cookies, http_cookies):
    bigcookies = pd.concat([jsc_cookies, js_cookies, http_cookies], ignore_index=True)
    return bigcookies

In [26]:
##### concatenate

In [7]:
# cookies (set/change httpresponce, js)
bigcookies = []
# experiment_num = sys.argv[0] #1
# web_nums = sys.argv[1] #3
experiment_num = 2
web_nums = 3
for i in range(1, web_nums+1):
    cookies = pd.read_csv("./{}-{}-ThirdCookies.csv".format(experiment_num, i))
    jscookiesc = extract_from_jscookies(cookies)
    httprequests = pd.read_csv("./{}-{}-httprequest.csv".format(experiment_num, i))
    httpc = extract_http(httprequests)
    js = pd.read_csv("./{}-{}-js.csv".format(experiment_num, i))
    jsc = extract_from_js(js)
    bigcookies.append(merge_cookies(jscookiesc, httpc, jsc))

/Users/rainyleon/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)
/Users/rainyleon/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:6130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [8]:
hugecookies = pd.concat(bigcookies)

In [9]:
hugecookies

,context_id,host,name,value,time_stamp,type
0,1,.demdex.net,demdex,47026235237090761000520117546690963371,2020-04-30T00:17:07.546Z,added-or-changed
1,1,.media.net,visitor-id,2312074272182012000V10,2020-04-30T00:17:07.603Z,added-or-changed
2,1,.scorecardresearch.com,UID,1A117223243a152c3341e7g1588205828,2020-04-30T00:17:08.439Z,added-or-changed
3,1,.scorecardresearch.com,UIDR,1588205828,2020-04-30T00:17:08.441Z,added-or-changed
4,1,.reddit.com,reddaid,RT36B44SRKAPJFIB,2020-04-30T00:17:09.504Z,added-or-changed
5,1,.rlcdn.com,rlas3,k2nnfz0Jcs/DVPE4+qiZpryOzCn6ZMU4wh5rdjmSbVA=,2020-04-30T00:17:09.814Z,added-or-changed
6,1,.rlcdn.com,pxrc,CAA=,2020-04-30T00:17:09.815Z,added-or-changed
7,1,.turn.com,uid,8541296435815272522,2020-04-30T00:17:09.861Z,added-or-changed
8,1,.rlcdn.com,rlas3,n9nAqGVUiBPDVPE4+qiZpryOzCn6ZMU4wh5rdjmSbVA=,2020-04-30T00:17:11.456Z,deleted
9,1,.rlcdn.com,rlas3,n9nAqGVUiBPDVPE4+qiZpryOzCn6ZMU4wh5rdjmSbVA=,2020-04-30T00:17:11.456Z,added-or-changed


In [10]:
hugecookies.type.unique()

array(['added-or-changed', 'deleted', 'sent', 'set', 'get'], dtype=object)

In [56]:
hugecookies.to_csv("./hugecookies/hugecookies-{}.csv".format(experiment_num), index=False)

##########
unique_hugecookies = hugecookies.drop_duplicates()
unique_hugecookies.to_csv("./hugecookies/unique_hugecookies-{}.csv".format(experiment_num), index=False)

In [57]:
hugecookies.shape

(67468, 6)

In [32]:
# unique_hugecookies.groupby("value").mean().shape

In [33]:
unique_context_1 = unique_hugecookies[unique_hugecookies.context_id == 1].value.tolist()
unique_context_2 = unique_hugecookies[unique_hugecookies.context_id == 2].value.tolist()
unique_context_3 = unique_hugecookies[unique_hugecookies.context_id == 3].value.tolist()

In [34]:
list(set(unique_context_1) & set(unique_context_2))

['',
 'AQvitULEwpGHyem3KD_sclC_TdevAkmkIDc5fX-2ctnM3cuySKT08zVqdm6IIk8PUxEDL9hvoMEK4titQ8kOr5jiWXjgwlFT-oo',
 '\\v1_Qy2edN3p3Xyn/PRyFq5qkA==\\',
 'SYYIIXTLSI2r3Pl2jBlm8Q',
 '7f675ea8-395e-4900-ac28-995748e08e07',
 '6ZmmRprj1JtqWW5~~1',
 '4679e76d-9062-4b75-8547-8ae6d7f4aaa1~~1',
 '00000000-0000-003a-398c-861588083038',
 'd=AQABBF45qF4CEEgvPTiD59Nt7NBRJgIJ_7MFEgEBAQGKqV6yXgAAAAAA_SMAAAcIXjmoXgIJ_7M&S=AQAAAjIjmhMSKUaUG3XfXfyVNH8',
 '5LD99aG83a85wM/A',
 'testWrite',
 '2fac0fa240ca1592',
 '1DA23a36a2aa20a318412601588083033',
 'di_9592bd7b1b884f9381548~~1',
 '4b07018c-0dcb-40e4-8314-5ae24eab86d9',
 'g_surferid~Xqg5XAAAAKg2BwFU',
 '4679e76d-9062-4b75-8547-8ae6d7f4aaa1',
 'YES',
 '1AB15E46-124E-4486-9D2E-CF149023BDE1',
 '1588083033',
 '4Oe53a4sDRVl1-qj0eXv6L',
 'NO_DATA',
 '203=bHHssFauB8Gn7J19HHvYAtfqgk-pvmGVbRLDCNL385XQIjO1-bGzE_cpYAp4bawqxjC6pYXp2udS6fZ4kOUxm8bq7wve-5WyJNPr-SNTYn1f2Z-75jTdVBxdQRzjMKPpaMjtRKafXCAhGP-IzdmcihVUgiW04OMAPMeMmcJZoAI',
 '00000000-0000-003a-3d76-c31588083153',
 'A

In [35]:
list(set(unique_context_1) & set(unique_context_3))

['',
 '1',
 'NO_DATA',
 'AHWqTUlOVZKvNy4CkZNqO7cRitxqXgMD91fWHMzrP_KlgTJoBV118EIpwWuHIXw7',
 '/']

In [36]:
list(set(unique_context_2) & set(unique_context_3))

['',
 '1',
 '0VKKhljkesAUbsROW..BeqDnl..F6o.1.0.BeqDnl.',
 'lax',
 'NO_DATA',
 'AHWqTUlOVZKvNy4CkZNqO7cRitxqXgMD91fWHMzrP_KlgTJoBV118EIpwWuHIXw7',
 '/']

In [37]:
list(set(unique_context_1) & set(unique_context_2) & set(unique_context_3))

['',
 '1',
 'NO_DATA',
 'AHWqTUlOVZKvNy4CkZNqO7cRitxqXgMD91fWHMzrP_KlgTJoBV118EIpwWuHIXw7',
 '/']

In [38]:
len(list(set(unique_context_1) & set(unique_context_2)))

57

In [39]:
len(list(set(unique_context_1) & set(unique_context_3)))

5

In [40]:
len(list(set(unique_context_2) & set(unique_context_3)))

7

In [63]:
# unique_hugecookies.groupby(["name", "value"]).value.unique()


# unique_hugecookies.groupby(["name", "value"]).value.unique().shape


(698,)